In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

<b><font color='#8B0000'>TASK</font></b>

Read the provided dataset and save it in the variable `dataset_exam`.

In [11]:
# YOUR CODE HERE

import datetime as dt

In [12]:
# HELPER CELL - RUN IT!

print("Helper cell - done. Please continue!")

Helper cell - done. Please continue!


## Tasks

1. Find the mean compleation rates across males and females.
2. Investigate why the mean completion rates are so low.
3. Find the correlation coefficients between the numerical variables.
4. Find what variable is the most correlated with out target variable.
5. Find the connection between birth rate and enrollment rate.
5. Document your results.

# Read our data

In [15]:
df = pd.read_csv('./input/Global_Education.csv', encoding='latin-1')

FileNotFoundError: [Errno 2] No such file or directory: './input/Global_Education.csv'

In [ ]:
df.info()

# Fix columns

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.replace(" ", "_")

In [ ]:
df.columns

# Drop unnecessary data

In [ ]:
df.drop(["Latitude_", "Longitude"], axis=1, inplace=True)

# Derive basic statistics

In [ ]:
statistics_df = df.describe()

In [ ]:
statistics_df

# Find number of rows and columns

In [ ]:
rows = df.shape[0]
cols = df.shape[1]

In [ ]:
print(f"Number of rows: {rows}, number of columns: {cols}")

# Find missing values

In [9]:
for col in df.columns:
    if df[col].isna().any():
        null_rate = df[col].isna().sum() / len(df[col])
        print(f'{col} null rate: {null_rate:.4f}')

NameError: name 'df' is not defined

# Deal with missing values

In [ ]:
for col in df.columns:
    if df[col].isna().any():
        mode = df[col].mode()[0]
        df[col] = df[col].fillna(mode)

# Find duplicates

In [ ]:
df.duplicated().any()

In [ ]:
drop_duplicates()

# New DataFrame

In [ ]:
completion_columns = df.columns[df.columns.str.startswith("Completion")]

In [ ]:
completion_columns

In [ ]:
completion_rates_by_country = df.groupby(by='Countries_and_areas')[completion_columns].mean()

In [ ]:
completion_rates_by_country

In [ ]:
completion_rates_by_country["Average_completion_rate"] = completion_rates_by_country.mean(axis=1)

In [ ]:
completion_rates_by_country

# Top 10 countries

In [ ]:
sorted_completion_rates = completion_rates_by_country.sort_values(by="Average_completion_rate", ascending=False)

In [ ]:
top_10_countries = sorted_completion_rates.head(10)
bottom_10_countries = sorted_completion_rates.tail(10)

In [ ]:
#top_10_countries = completion_rates_by_country.nlargest(10, "Average_completion_rate")
#bottom_10_countries = completion_rates_by_country.nsmallest(10, "Average_completion_rate"))

In [ ]:
plt.barh(top_10_countries.index.values[::-1], top_10_countries["Average_completion_rate"][::-1])
plt.title("Top 10 countries by average completion rate")
plt.show()

In [ ]:
male_columns_completion = df.columns[(df.columns.str.startswith("Completion")) & (df.columns.str.endswith("Male"))]
female_columns_completion = df.columns[(df.columns.str.startswith("Completion")) & (df.columns.str.endswith("Female"))]

In [ ]:
male_columns_completion

In [ ]:
df[male_columns_completion].mean()

In [ ]:
df[female_columns_completion].mean()

# Discover outliers

In [ ]:
df['Completion_Rate_Primary_Male'].value_counts()

In [ ]:
df[female_columns_completion].mean()

In [ ]:
df['Completion_Rate_Primary_Female'].value_counts()

We observed that average completion rate is oddly low. Therefore, we plot our variables to find out what is happening.

In [ ]:
female_completion_rate_scores = df[female_columns_completion[0]].value_counts().index.values
female_completion_rate_count = df[female_columns_completion[0]].value_counts().values

In [ ]:
plt.scatter(female_completion_rate_scores, female_completion_rate_count)
plt.title("Distribution of female completion rate (primary)")
plt.xlabel("Score")
plt.ylabel("Frequency")
plt.show()

In [ ]:
#numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#df.select_dtypes(include=numerics)

# Numerical columns

In [ ]:
numerical_df = df.select_dtypes(include='number')

In [ ]:
numerical_df.info()

# Correlation between variables

In [ ]:
corr_matrix = numerical_df.corr()
corr_matrix

In [ ]:
# try annot=True
sns.heatmap(corr_matrix)
plt.show()

# Unemployment drivers

In [ ]:
corr_matrix['Unemployment_Rate'].sort_values(ascending=False)

In [ ]:
absolute_unemployment_drivers = corr_matrix['Unemployment_Rate'].abs().sort_values(ascending=False)
absolute_unemployment_drivers

In [ ]:
plt.scatter(df["Unemployment_Rate"], df["Completion_Rate_Lower_Secondary_Female"])
plt.title("Unemployment rate VS Completion rate of secondary education (female)")
plt.xlabel("Unemployment_rate")
plt.ylabel("Completion_Rate_Lower_Secondary_Female")
plt.show()

In [ ]:
#absolute_unemployment_drivers.drop("Unemployment_Rate", inplace=True)

In [ ]:
top_5_drivers = absolute_unemployment_drivers.head(6).index.values
top_5_drivers

In [ ]:
top_5_corr_matrix = df[top_5_drivers].corr()

In [ ]:
sns.heatmap(top_5_corr_matrix, annot=True)
plt.show()

# Unemployment by country

In [ ]:
mean_unemployment_by_country = df.groupby(by='Countries_and_areas')['Unemployment_Rate'].mean()

In [ ]:
top_15_countries = mean_unemployment_by_country.nlargest(n=15)
top_15_countries

In [ ]:
color_map = ['#9ad6aa' for col in range(15)]
color_map[0] = color_map[1] = color_map[2] = '#32a852'

In [ ]:
plt.figure(figsize=(9, 7), dpi=300)
plt.barh(top_15_countries.index.values[::-1], top_15_countries.values[::-1], color=color_map[::-1])
plt.title("Top 15 countries by unemployment rate")
plt.xlabel("Unemployment rate")
plt.show()

# Birth rate and enrollment

In [ ]:
plt.figure(layout='constrained')
plt.scatter(df.Birth_Rate, df.Gross_Tertiary_Education_Enrollment, label='Gross_Tertiary_Education_Enrollment')
plt.scatter(df.Birth_Rate, df.Gross_Primary_Education_Enrollment, label='Gross_Primary_Education_Enrollment')
plt.title("Birth rate VS enrollment")
plt.xlabel("Birth rate")
plt.legend(loc='upper left')
plt.show()

# Documentation